In [ ]:
# Lets start by detecting if we can detect text in the image using tessaract 

In [4]:
import pytesseract
import PIL
from PIL import Image


In [5]:
# testing image a1 for OCR 
img = Image.open('readonly/small_img/a-0.png')
img.show()

In [7]:
text = pytesseract.image_to_string(img) 
if "Christopher" in text:
    print("found")


found


In [8]:
# lets test all images in file for text "Christopher"

for i in range(4):
    filename = f'readonly/small_img/a-{i}.png'
    img = Image.open(filename)
    text = pytesseract.image_to_string(img)
    if "Christopher" in text:
        print("File :" , filename)
        print("found")
    


File : readonly/small_img/a-0.png
found
File : readonly/small_img/a-3.png
found


In [10]:
# above output shows OCR is wroking without kraken, no need to complicate the code. Its showing Christopher is in image a1 and a3. 
# lets search for mark in large file 
# mark should be in files 0,1,2,3,8,10,13 only 
for i in range(14):
    filename = f'readonly/images/a-{i}.png'
    img = Image.open(filename)
    print(f'processing file : a-{i}.png')
    text = pytesseract.image_to_string(img)
    if "Mark" in text:
        print("Mark found")
    else:
        print("Mark not in file")
    print("----------------------------------------------------")
    
 
    


processing file : a-0.png
Mark found
----------------------------------------------------
processing file : a-1.png
Mark found
----------------------------------------------------
processing file : a-2.png
Mark found
----------------------------------------------------
processing file : a-3.png
Mark found
----------------------------------------------------
processing file : a-4.png
Mark not in file
----------------------------------------------------
processing file : a-5.png
Mark not in file
----------------------------------------------------
processing file : a-6.png
Mark not in file
----------------------------------------------------
processing file : a-7.png
Mark not in file
----------------------------------------------------
processing file : a-8.png
Mark found
----------------------------------------------------
processing file : a-9.png
Mark not in file
----------------------------------------------------
processing file : a-10.png
Mark found
--------------------------------

In [14]:
# Output shows that mark found in required files only. So no need to use kraken , tessaract is working fine.  

# Now have to extract the face images from each image and make a dictionary 

from PIL import ImageDraw
from PIL import Image
import cv2 as cv

cv_img = cv.imread('readonly/small_img/a-0.png')
face_cascade = cv.CascadeClassifier(
    'readonly/haarcascade_frontalface_default.xml')



In [15]:
cv_img_bin = cv.threshold(cv_img,120,255,cv.THRESH_BINARY)[1]
#  OpenCV is return the coordinates as (x,y,w,h), 
#  while PIL.ImageDraw is looking for (x1,y1,x2,y2).

face_coordinates = face_cascade.detectMultiScale(cv_img_bin)
# Now lets see the results
file_with_faces = Image.open('readonly/small_img/a-0.png')



In [ ]:
print(face_coordinates)

In [ ]:
display(crop_faces(face_coordinates,file_with_faces))

In [ ]:
# Generate face coordinates from an Image given threshold
def face_coordinate_generator(filepath, threshold):
    '''generates coordinates for a face in a given image 
    :param filepath: File dir path for the image file
    :param threshold: threshold value for binarization of file for face detection, out of (0,255)
    :return: ndarray of face coordinates in (x,y,w,h) format
    '''
    import cv2 as cv

    cv_img = cv.imread(filepath)
    face_cascade = cv.CascadeClassifier(
        'readonly/haarcascade_frontalface_default.xml')
    cv_img_bin = cv.threshold(cv_img, threshold, 255, cv.THRESH_BINARY)[1]
    #  OpenCV is return the coordinates as (x,y,w,h),
    face_coordinates = face_cascade.detectMultiScale(cv_img_bin)

    return face_coordinates


In [ ]:
# function to generate contact sheet of all faces from a given PIL image and face coordinates
def contactsheet_faces(face_coordinates, file_with_faces):
    '''given face coordinates and file on which faces are, returns a contactsheet containg all the faces in the file
    :param face_coordinates: A ndarray with all the face coordinates generated from openCV 
    :param file_with_faces: PIL Image on which faces are
    :return : a PIL image containing all the faces 
    '''
    pil_img = file_with_faces
    allfaces = []

    MAX_SIZE = (100, 100)  # thumbain size (width,length)
    for x, y, w, h in face_coordinates:

        single_face = pil_img.crop((x, y, x+w, y+h))

        single_face.thumbnail(MAX_SIZE)
        allfaces.append(single_face)

    if len(allfaces) == 0:
        print("NO FACES IN FILE")
        return None

    # create a contact sheet
    images_in_row = 5
    contact_sheet = PIL.Image.new(mode=allfaces[0].mode,
                                  size=(MAX_SIZE[0]*images_in_row, (MAX_SIZE[1]*((len(allfaces)//images_in_row)+1))))
    # Pasting images on contact_sheet
    x = 0
    y = 0

    for img in allfaces:
        # Lets paste the current image into the contact sheet
        contact_sheet.paste(img, (x, y))
        first_image = allfaces[0]
        # Now we update our X position. If it is going to be the width of the image, then we set it to 0
        # and update Y as well to point to the next "line" of the contact sheet.
        if x+first_image.width == contact_sheet.width:
            x = 0
            y = y+first_image.height
        else:
            x = x+first_image.width

    return contact_sheet
